### 활성화 함수 계층 구현하기 

ReLU와 Sigmoid 계층을 구현한다

#### ReLU

순전파 때의 입력인 x가 0보다 크면 역전파는 상류의 값을 그대로 하류로 흘린다. 반면, 순전파 때 x가 0이하면 연전파 때는 하류로 신호를 보내지 않는다.


이를 구현한 코드는 아래와 같다.


In [5]:
import numpy as np

class Relu:
    def __init__(self):
        self.mask=None
        
    def forward(self,x):
        self.mask=(x<=0)
        out = x.copy()
        out[self.mask]=0
        
        return out
    
    def backward(self,dout):
        dout[self.mask]=0
        dx = dout
        
        return dx
    
# Relu 클래스는 mask라는 인스턴스 변수를 가지고, mask는 True/False로 구성된 넘파이 배열로, 순전파의 입력인 x의
# 원소값이 0 이하인 인덱스는 True, 그 외(0보다 큰 원소)는 False로 유지한다. 예컨대 mask 변수는 다음 예와 같이 True/False로
# 구성된 넘파이 배열을 유지한다.






        

In [8]:
x=np.array([[1.0,-0.5],[-2.0,3.0]])
print(x)

mask=(x<=0)

print(mask)




[[ 1.  -0.5]
 [-2.   3. ]]
[[False  True]
 [ True False]]


순전파 때의 입력 값이 0이하면 역전파 때의 값은 0이 돼야 하고, 
역전파 때는 순전파 때 만들어둔 mask를 써어 mask의 원소가 True인 곳에는 상류에서
전파된 dout을 0으로 설정한다

- ReLU 계층은 전기 회로의 '스위치'에 비유할 수 있다. 순전파 때 전류가 흐르고 있으면 스위치를 ON으로 하고, 흐르지 않으면 OFF로 한다. 역전파 때는 스위치가 ON이라면 전류가 그대로 흐르고, OFF면 더 이상 흐르지 않는다



### Sigmoid 계층

node의 구성으로는 +,-,/,exp가 있다.

* exp노드는 y=exp(x) 계산을 수행하고, '/'노드는 y=1/x 계산을 수행한다.

* '/'노드의 역전파는 -y^2(순전파의 출력을제곱한 후 마이너스를 붙인 값)을 곱해서 하류로 전달한다. 

* exp노드는 y=exp(x) 연산을 하고, 역전파에서는 출력값을(y) 하류로 전파한다.



이를 다 종합하면 sigmoid의 역전파는 다음과 같이 정의할 수 있다.

                     backpropagation
(roundoL/roundoy)         --->           (roundoL/roundoy)*y^2*exp(-x)

이를 정리하면 ..

#### (roundoL/roundoy)*y^2*exp(-x)     --->    (roundoL/roundoy)y(1-y)
 
 
 




In [2]:
class Sigmoid:
    
    def __init__(self):
        self.out=None
        
    def forward(self,x):
        out=1/(1+np.exp(-x))
        self.out=out

        return out
    
    
    def back(self,x):
        dx=dout*(1.0-self.out)*self.out
        
        return dx
    
    


### Affine layer

신경망의 순전파 때 수행하는 행렬의 곱은 기하학에서는 어파인 변환 이라고 한다.





지금까지의 계산 그래프는 노드 사이에 '스칼라값'이 흘렀는 데 반해, 행렬이 흐르고 있다.



In [6]:
import numpy as np

X_dot_W=np.array([[0,0,0],[10,10,10]])

B=np.array([1,2,3])

X_dot_W

X_dot_W+B




array([[ 1,  2,  3],
       [11, 12, 13]])

In [7]:
class Affine:
    def __init__(self):
        self.W=W
        self.b=b
        self.x=None
        self.dW=None
        self.db=None
        
    def forward(self,x):
        self.x=x
        out = np.dot(x,self.W)+self.b
        
        return out
    
    def back(self,dout):
        
        dx=np.dot(dout,self.W.T)
        self.dW=np.dot(self.x.T,out)
        self.db=np.sum(dout,axis=0)
        
        return dx
    
    
        

In [8]:
p=np.array([[2,3,4],[3,4,5]])

### softmax-with-Loss layer

softmax는 입력 값을 정규화하여 출력한다. 

입력 이미지가 Affine 계층과 ReLU 계층을 통과하여 변환되고, 마지막 Softmax 계층에 의해서 10개의
입력이 정규화 된다.

##### 신경망에서 수행하는 작업은 학습과 추론이다. 추론할 때는 일반적으로 Softmax 계층을 사용하지 않는다. 추론할 때는 마지막 Affine 계층의 출력을 인식 결과로 이용한다. 또한, 신경망에서 정규화하지 않는 출력 결과에서는 softmax앞의 Affine 계층의 출력을 score라고 한다. 즉,신경망 추론에서 답을 하나만 내는 경우에는 가장 높은 점수만 알면 되니 softmax는 필요가 없기 때문. 반면 신경망을 학습할 때는 Softmax계층이 필요하다.



### CEE,Softmax간 backpropagation에 대해서는 그림과 책을 참조하는게 좋을듯!


* 'softmax 함수'의 손실 함수로 '교차 엔트로피 오차'를 사용하니 역전파가 (y1-t1,y2-t2,y3-t3)로 말끔히 떨어진다.
* 회귀의 출력층에서 사용하는 '항등 함수'의 손실 함수로 '오차제곱합'을 이용하는 이유도 이와 같다.
* 즉, '항등 함수'의 손실 함수로 '오차제곱합'을 사용하면 역전파의 결과가 위처럼 말끔히 떨어진다.



In [14]:
#Softmax-with-loss


class SoftmaxWithLoss:
    def __init__(self):
        self.loss=None
        self.y=None
        self.t=None
        
    def forward(self,x,t):
        
        self.t = t
        self.y = softmax(x)
        self.loss=cross_entropy_error(self.y,self.t)
        return self.loss
    
    def backword(self,dout=1):
        batch_size=self.t.shape[0]
        dx=(self.y-self.t)/batch_size
        
        return dx
    